# 인스타 데이터 수집_v2

## 외부지표
(프로필 정보 관련) ID, 게시물 수, 팔로워 수, 팔로우 수, 카테고리, 자기소개  
(각 게시물 정보 관련) 좋아요 수, 댓글 수  
(보류) 게시글 내용, 댓글 내용  

## 추가한 기능
- sample_.csv 파일을 불러들여 각 url과 인플루언서 선정 변수 값을 받을 수 있게 코드 구성.
- 게시물 수 만큼 행 데이터를 만들어서 csv 파일로 도출.
- 이미지 데이터들을 로컬 컴퓨터에 저장하는 기능 추가.

## 코드 참고 사항
### 좋아요 개수 추출에서 생기는 문제
사용자의 좋아요 수를 공개 여부에 따라 선택자 명과 태그 구조가 달라집니다.  
최대한 타협안으로 좋아요 개수 전체 태그의 클래스 명을 사용했습니다.  
그래서 좋아요 개수뿐만 아니라 글자도 같이 포함되는데 이는 파이썬 전처리로 해결해야 될 것 같습니다.  

- 좋아요 개수 공개했을 때 사용할 선택자 명: "._ae5m .html-span"  
- 좋아요 개수 공개하지 않을 때 사용할 선택자 명: "._ae5m a:nth-child(2)"  
- 이 코드에서는 상위 태그인 "._ae5m .x193iq5w.xeuugli"를 사용했습니다.

### 로딩 시간에 따른 오류 발생 가능
로딩 시간 문제로 webdriver가 선택자를 잘 파악하지 못할 수도 있습니다.  
일단 selenium webdriver 함수 동작 이후마다 대기 시간을 줌으로써 오류를 줄여보았습니다.

## 실행 시 주의!!
- 크롬 드라이버를 다운받고 이 코드 파일와 동일한 경로에 넣어 주세요.
- 이 코드 창을 브라우저 크기를 줄이고 코드 실행해야 잘 작동합니다.  

## 1. 세팅

In [ ]:
# !pip install selenium
# !pip install beautifulsoup4
# !pip install webdriver_manager
# !pip install pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests

import time
import random

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버 자동 업데이트을 위한 모듈

import urllib.request # 이미지 저장 라이브러리

import csv
import pandas as pd

In [ ]:
# sample_.csv 불러오기
sample = pd.read_csv('sample_.csv')
sample

In [ ]:
# 필요한 열만 추출
subset = sample[['check', 'link']]
subset

In [ ]:
checks = list(subset['check'])
links = list(subset['link'])

In [ ]:
# 크롬 드라이버 옵션 관련 코드

# 브라우저 꺼짐 방지 옵션
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('window-size=1920x1080') # pc용 사이즈
options.add_argument('--start-maximized')     # 브라우저가 최대화된 상태로 실행

# 크롤링 차단 방지를 위한 user-agent 기입 (접속에 오류가 날 경우)
# options.add_argument("user-agent=본인의user-agent")
# user-agent=는 띄어쓰지 말고 꼭 붙여쓰기

# 불필요한 에러 메시지 삭제
options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 드라이버 최신 버전 설정
service = Service(executable_path=ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

driver 객체의 page_source라는 인스턴스 변수: 웹 서버로부터 전달받은 HTML 코드가 저장되어 있음.  

1. selenium에서 page_source를 사용하여 해당 페이지의 소스, html을 가져오고 
2. 가져온 html을 BeautifulSoup으로 넘겨줌.
   
이를 통해 페이지 이동 및 동작은 selenium으로 크롬 드라이버를 조작하여 할 수 있고,  
원하는 페이지로 이동후 html을 BeatifulSoup으로 전달하여 원하는 웹 데이터를 가져와 사용할 수 있다.

In [ ]:
# 인스타그램 접속
driver.get('https://instagram.com')

# 인스타그램 자동 로그인
time.sleep(2)
login_id = driver.find_element(By.CSS_SELECTOR, 'input[name="username"]')
login_id.send_keys('본인의 인스타 아이디 입력')
driver.implicitly_wait(10) # 나타날 때까지 최대 10초동안 기다리기
login_pwd = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
driver.implicitly_wait(10)
login_pwd.send_keys('본인의 인스타 비밀번호 입력')
time.sleep(2)
login_id.send_keys(Keys.ENTER)
time.sleep(3)

## 2 & 3. 데이터 수집 및 csv 파일 저장

### 2. 프로필 관련 정보: 팔로워, 팔로잉, 게시물 수, 닉네임, 자기소개
- bs4로 html문서를 읽어 수집

### 3. 각 게시물 관련 정보: 좋아요 수, 댓글 수  
-  셀레니움으로 게시물을 클릭해 이동하며 수집

In [ ]:
# 필드명 지정
fieldnames = ['id', 'media_count', 'followers_count', 'follows_count', 'category', 'introduction', 'main_img', 'like_count', 'check']

In [ ]:
# crawling.csv 라는 파일 생성하고 상단에 컬럼 이름 적기
f = open('crawling.csv', 'w', newline='', encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=fieldnames)
writer.writeheader()

# 반복문으로 각 인플루언서 주소에 접속
for j in range(4): # 상위부터 4개까지의 계정을 가져옴

    # sample_에서 추출한 데이터 가져오기
    link = links[j]   # 해당 순서 사용자의 주소
    check = checks[j] # 해당 순서 사용자의 인플루언서 여부

    ##### 2. 프로필 관련 정보 가져오기 #####
    driver.get(links[j])             # 페이지 이동
    time.sleep(random.uniform(3, 4)) # 설정한 초 간격 사이로 랜덤하게 쉬기
    
    html = driver.page_source # html(페이지 소스) 가져오기
    time.sleep(random.uniform(3, 4))
    
    soup = BeautifulSoup(html)
    time.sleep(random.uniform(2, 3))

    id = soup.select_one('h2').text # 사용자 id
    
    profile = soup.select("._ac2a > span")
    media_count = profile[0].text          # 게시물 수
    followers_count = profile[1].text      # 팔로워 수
    follows_count = profile[2].text        # 팔로우 수 
    
    try: 
        category = soup.select_one("._aacy").text     # 카테고리
    except:
        category = 'NA' # 존재하지 않음을 표기
    
    try: 
        introduction = soup.select_one("._aacx").text # 자기소개
    except:
        introduction = 'NA'

    ##### 3. 각 게시물들의 대표 이미지와 좋아요 수 가져오기 #####
    # 첫번째 게시물 클릭
    time.sleep(random.uniform(1, 2))
    driver.find_element(By.CSS_SELECTOR, '._aagu').click()
    driver.implicitly_wait(10)
    
    # 첫번째 게시물부터 데이터를 추출하고 다음 게시물을 클릭
    for i in range(5): # 이 숫자를 조절하면 원하는 게시물 수만큼 가져올 수 있음
        
        # 예외처리로 일단 사진만 가져오게 하고 영상은 지나치도록 함
        try:
            # 대표 이미지 가져오기
            main_img = driver.find_element(By.CSS_SELECTOR, "._aagu._aato img").get_attribute("src")
            #print(f'{id}의 {i}번째 게시물 이미지: ' + main_img)
    
            # 좋아요 수
            like_count = driver.find_element(By.CSS_SELECTOR, "._ae5m .x193iq5w.xeuugli").text # 좋아요 수 영역
            #print(f'{id}의 {i}번째 게시물 좋아요 수: ' + like_count)
    
            driver.find_element(By.CSS_SELECTOR, '._aaqg._aaqh').click() # 다음 버튼 클릭
            time.sleep(random.uniform(1, 2))
    
        except:
            # 영상은 지나치기 
            main_img = 'NA'
            #print(f'{id}의 {i}번째 게시물' + "은 video입니다.")
            
            # 좋아요 수
            like_count = driver.find_element(By.CSS_SELECTOR, "._ae5m .x193iq5w.xeuugli").text
            #print(f'{id}의 {i}번째 게시물 좋아요 수: ' + like_count)
    
            driver.find_element(By.CSS_SELECTOR, '._aaqg._aaqh').click() # 다음 버튼 클릭
            time.sleep(random.uniform(1, 2))
    
        #print()

        # 각 컬럼에 해당하는 값들을 기입해 하나의 행 데이터를 만들기
        writer.writerow({'id': id, 'media_count': media_count, 'followers_count': followers_count, 'follows_count': follows_count, 
                         'category': category, 'introduction': introduction, 'main_img': main_img, 'like_count': like_count, 'check': check})
        time.sleep(random.uniform(1, 2))
    
    # 게시물 창 닫기
    driver.find_element(By.CSS_SELECTOR, '.x160vmok line').click()
    time.sleep(random.uniform(2, 3))
    
# csv 파일 작성 끝나면 닫기
f.close()

## 4. 이미지를 로컬 컴퓨터에 저장

In [ ]:
# 만든 crawling.csv 불러오기
result = pd.read_csv('crawling.csv')
result

In [ ]:
img_ls = list(result["main_img"])
img_ls

In [ ]:
# 크롤링 이미지 파일 저장
for k in range(len(img_ls)):

    # 이미지 경로가 존재하지 않을 경우 -> 저장하는 단계 없이 넘어가기 
    if type(img_ls[k]) == float: # 주의: nan값의 타입이 float임
        print(f"{k}번째 행은 이미지가 존재하지 않음")
        continue

    # 이미지 경로가 존재할 경우 -> 아마지 저장하기 (저장명은 행 인덱스 번호로 지정)
    urllib.request.urlretrieve(img_ls[k], "이미지를 저장할 폴더 경로" + "0"*(3-len(str(k)))+str(k) + ".jpg") 
    # .urlretrieve(저장할 이미지의 원래 경로, 내 컴퓨터에 저장할 경로의 이름)